In [1]:
import pandas as pd
%matplotlib inline
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *

import warnings; warnings.simplefilter('ignore')
from collections import Counter
from scipy.sparse.linalg import svds



In [32]:
# HELPING FUNCTIONS

stemmer = PorterStemmer()

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def clean_(sen):
    
    sen = re.sub(r'[^a-zA-Z\.\:\@0-9_ ]+', '', sen) 
    sen = sen.lower()
    sen = sen.split()
#     sen = [lemmatize_stemming(x) for x in sen]
    sen = ' '.join(sen)
    
    return sen



def get_events(metadata):
#     metadata = pd.read_excel(path)
    metadata['Type']=[ x.lower() for x in metadata['Type']]
        
    eventname= metadata['Event name'].fillna('')
    description = (metadata['Event time'] + metadata['Venue']+metadata['Description']+metadata['Source']+metadata['Type']+metadata['City']).fillna('')
    description_clean= list(description)

    i=0
    while(i< len(description)):
        description_clean[i]= clean_(description[i])
        i+=1
    
    #improve Description
    metadata['Description']=description_clean
    return metadata


In [33]:
#APPLICATION DATA

#INPUT FOR THE APPLICATION!
def get_prep_data(input_user_id):
    data=pd.read_excel('activities.xls')

    df_users = pd.DataFrame(columns= ['userId','userName'])
    # df_events = get_events("activities.xls")
    df_events=get_events(data)

    m = pd.read_csv('movies/movies.csv')

    df_events['eventId'] = m['movieId']

    # df_old_events=df_events.iloc[0:int(df_events.shape[0]*0.8),:]
    # df_new_events=df_events.iloc[int(df_events.shape[0]*0.8):,:]

    df_rating = pd.read_csv("movies/ratings.csv")
    df_rating.rename(columns={'movieId':'eventId'},inplace= True)

    userRated=df_rating.loc[df_rating['userId']==input_user_id]

    df_old_events=df_events.loc[df_events['eventId'].isin(list(userRated['eventId']))] #Contains user input
    df_new_events=df_events.loc[~df_events['eventId'].isin(list(userRated['eventId']))] #Does Not contain user input
    return data,df_events,df_rating,df_old_events,df_new_events


In [5]:
# def get_frequency(df,to_check,col_name,no):
#     if int(df.shape[0]>0):
#         n=no
#         #count
#         tmp_count = Counter(list(df[to_check]))
#         tmp = pd.DataFrame.from_dict(tmp_count, orient='index')

#         #sort
#         tmp=tmp.sort_values(0,ascending=False)

#         #remove index
#         tmp.reset_index(level=0, inplace=True)

#         #rename
#         tmp=tmp.rename(columns={"index":col_name,0:"Frequency"})

#         #get Top N----
#         tmp=tmp.loc[0:n]

#         #Perc-Freq
#         sumed=sum(tmp["Frequency"])
#         tmp["Percentage of total"]=(tmp["Frequency"]/sumed)*100
#         return tmp
#     else:
#         print("DataFrame is empty")
        
# get_frequency(df_rating,'userId','userId',10)

In [6]:
df_new_events=df_new_events.loc[df_new_events['eventId'].isin(list(df_new_events['eventId'].unique())[:100])]
df_new_events

,Event name,Event time,Venue,Description,Source,Picture,Type,City,Rated,eventId
3,Visual Design 101,\t\t\t\t\t\tWed Dec 11 2019 at 03:00 pm to 05:...,"National Incubation Center, Plot 24-b, H-9/1,...",wed dec 11 2019 at 03:00 pm to 05:00 pm nation...,National Incubation Center,https://cdn-az.allevents.in/events8/banners/f9...,entertainment,islamabad,False,4
8,3 Days Trip To Swat Valley And Malam Jaba,\t\t\t\t\t\tFri Dec 13 2019 at 07:00 am to Sun...,"I-8/1, Islamabad, Pakistan",fri dec 13 2019 at 07:00 am to sun dec 15 2019...,North Adventure Club,https://cdn-az.allevents.in/events8/banners/0f...,entertainment,islamabad,False,9
11,سنی ان سنی’ The Wound Cannot be Heard,\t\t\t\t\t\tFri Dec 13 2019 at 07:00 pm to 09:...,"ChaiLogue, Shop number 3, Imperial Square, Kh...",fri dec 13 2019 at 07:00 pm to 09:00 pm chailo...,ChaiLogue,https://cdn-az.allevents.in/events5/banners/31...,entertainment,islamabad,False,12
12,Snow Hike Miranjani Top,\t\t\t\t\t\tSat Dec 14 2019 at 07:00 am to 10:...,"Ultra Adventure Club, Bluearea, Islamabad, Pa...",sat dec 14 2019 at 07:00 am to 10:00 pm ultra ...,Ultra Adventure Club,https://cdn-az.allevents.in/events6/banners/bd...,entertainment,islamabad,False,13
13,Certified Self-Discovery & Career Counseling W...,\t\t\t\t\t\tSat Dec 14 2019 at 01:00 pm to 04:...,"LaunchPad7, 1st floor Al Rehman Chambers Faza...",sat dec 14 2019 at 01:00 pm to 04:00 pm launch...,Mind Empowerment Consultancy,https://cdn-az.allevents.in/events2/banners/b0...,entertainment,islamabad,False,14
...,...,...,...,...,...,...,...,...,...,...
203,Developers Weekend,\t\t\t\t\t\tSat Feb 22 2020 at 08:00 am to Sun...,"COMSATS, Park Road, Chak Shahzad, Islamabad, ...",sat feb 22 2020 at 08:00 am to sun feb 23 2020...,"Computer Science Society, CUI, Islamabad",https://cdn-az.allevents.in/events7/banners/5f...,workshops,islamabad,False,237
204,Students Package: Karachi to Kaghan Shogran & ...,\t\t\t\t\t\tFri Dec 06 2019 at 08:00 pm to Thu...,"Surti Tours and Travels, b-19 noor housing pr...",,NaN,https://cdn5.allevents.in/images/dates/6-DEC_2...,entertainment,karachi,False,238
205,Thinking Films: Parasite (2019),\t\t\t\t\t\tWed Dec 11 2019 at 07:00 pm to 09:...,"T2F, 10-C, Sunset Lane 5, Phase 2 Extension, ...",wed dec 11 2019 at 07:00 pm to 09:30 pm t2f 10...,The Second Floor (T2F),https://cdn-az.allevents.in/events1/banners/4e...,entertainment,karachi,False,239
206,The Extractor's Mania,\t\t\t\t\t\tThu Dec 12 2019 at 11:00 am to 02:...,"Dreamworld Club, Gulshan-e-Maymar, Karachi, P...",thu dec 12 2019 at 11:00 am to 02:00 pm dreamw...,Metallurgical Engineers' Society - MES NED,https://cdn-az.allevents.in/events7/banners/0e...,entertainment,karachi,False,240


In [7]:
# HELPING FUNCTIONS

stemmer = PorterStemmer()

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def clean_(sen):
    
    sen = re.sub(r'[^a-zA-Z\.0-9 ]+', '', sen) 
    sen = sen.lower()
    sen = sen.split()
#     sen = [lemmatize_stemming(x) for x in sen]
    sen = ' '.join(sen)
    
    return sen



def get_events(path):
    metadata = pd.read_excel(path)
    metadata['Type']=[ x.lower() for x in metadata['Type']]
    
    eventname= metadata['Event name'].fillna('')
    description = (metadata['Event time'] + metadata['Venue']+metadata['Description']+metadata['Source']+metadata['Type']+metadata['City']).fillna('')
    description_clean= list(description)

    i=0
    while(i< len(description)):
        description_clean[i]= clean_(description[i])
        i+=1
    
    #improve Description
    metadata['Description']=description_clean
    return metadata

    

def first_element(x):
    return x[1]

# def get_recommendations(title,headings,mapping_,n=10):
#     """
#     Finds 10 most similar events using the pairwise cosine similarity.
#     """
#     index = (mapping_[title])
# #     print(index)
    
#     simi = list((cosine_similarity[index]))
#     indexes = list(range(len(simi)))
#     simi= list(zip(indexes,simi))
# #     print (sim_scores)
    
#     simi = sorted(simi, key=first_ele, reverse=True)
    
# #     ignorning first= itself
#     simi=simi[1:]
    
#     recommendations = metadata.iloc[[i[0] for i in simi]][headings]
    
#     return recommendations.head(n)


def get_recommendations_pop(metadata,title,mapping): #popularity based context
    
    index = (mapping[title])
    simi = list((cosine_similarity[index]))
    indexes = list(range(len(simi)))
    simi= list(zip(indexes,simi))
    simi = sorted(simi, key=first_ele, reverse=True)
    
    
    simi = simi[1:60]
    
    #calculate vote_count and avergare before doing this
    recommendations = metadata.iloc[[i[0] for i in simi]][['title', 'vote_count', 'vote_average']]
    
    items = metadata.iloc[[i[0] for i in simi]]
    
    vote_counts = items[items['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = items[items['vote_average'].notnull()]['vote_average'].astype('float')
    
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    
    
    recommendations = items[(items['vote_count'].notnull()) & (items['vote_average'].notnull()) & (items['vote_count'] >= m)]
    
    recommendations['vote_count'] = recommendations['vote_count'].astype('int')
    recommendations['vote_average'] = recommendations['vote_average'].astype('float')
    
    recommendations['weighted_rating'] = recommendations.apply(weighted_rating,args=(C,m), axis=1)
    recommendations = recommendations.sort_values('weighted_rating', ascending=False)
#     print (recommendations.head(2))
    recommendations=recommendations[['title', 'vote_average']]
    recommendations=recommendations.head(10)
    return recommendations


def weighted_rating(x,C,m):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)



# def recommend_eventss(userID, preds_matrix , items_matrix, rating_matrix, n=10):
    
    
#     user_predictions = preds_matrix.loc[userID]
#     print(user)
    
    
# #     user_rating = rating_matrix.loc[userID]
    
# #     ratings = user_rating.merge(items_matrix, how = 'left', left_on = 'itemId', right_on = 'ItemID')
# #     ratings=ratings.sort_values('rating', ascending = False)
    
# #     items_matrix=items_matrix.dropna()

# #     recommendations = items_matrix.merge(pd.DataFrame(user_predictions), how = 'left',left_on = 'ItemID',right_on = 'itemId')
   
# #     recommendations=recommendations.drop(columns=['Genres'])
# #     recommendations=recommendations.sort_values(userID - 1, ascending = False)
# #     recommendations=recommendations.rename(columns = {userID - 1: 'Predictions'})
                    

#     return recommendations.head(n)



In [8]:
#inputs to below function: input_user_id,df_rating,df_events,n= 6

In [14]:
# n=6
input_user_id=414
def get_similar_users(input_user_id,df_rating,df_events,n= 6):
    new_ratings = pd.merge(df_rating,df_events[['Event name','eventId']],left_on='eventId',right_on='eventId')
#     new_ratings = pd.merge(df_events[['Event name','eventId']],df_rating,left_on='eventId',right_on='eventId')

    #zero means that user has never voted for this item.
    rating_matrix = new_ratings.pivot(index='userId',columns='eventId',values='rating').fillna(0)
    
#     print(rating_matrix)
    knn = NearestNeighbors(metric='cosine',algorithm='brute')
    knn.fit((rating_matrix.to_numpy()))

    userid = input_user_id

    similar_user =[]

    # (excluded itself)
    value = rating_matrix.loc[ userid,:].values
    value = value.reshape(1,-1)
    print(rating_matrix.shape,'=====',np.shape(value))
    distances, indices = knn.kneighbors(value,n_neighbors=n) 
    distances=distances.flatten()
    indices=indices.flatten()

    print("Five Nearest Neighbors of user{0}:".format(userid))

    for i in range(1,len(distances)):
        similar_user.append(rating_matrix.index[indices[i]])
    print('about to return! ',similar_user)
    return similar_user


In [36]:
input_user_id=414
min_active=1000000
rec=100

#get data
data,df_events,df_rating,df_old_events,df_new_events=get_prep_data(input_user_id)
#main function
def system(input_user_id,df_new_events, df_old_events,df_rating,min_active_users=min_active,recent=rec):
    app_active_users = []
    
    for n_e in range(len(df_new_events)):
        if df_new_events.iloc[n_e]['eventId'] in df_rating['eventId']:
            app_active_users.extend( list(df_rating[df_rating['eventId'] == df_new_events.iloc[n_e]['eventId'] ]['userId'])  )
    app_active_users=np.unique(np.array(app_active_users))
    #Post cold start        
    similar_users_id=[]
    if len(app_active_users)> min_active_users:
        app_active_users=np.append(app_active_users,input_user_id)

        df_subset_ratings = df_rating[df_rating.userId.isin(app_active_users)]

        similar_users_id = get_similar_users(input_user_id,df_subset_ratings, df_old_events)
        similar_users_id.append(input_user_id)
        
        #get older user data.
        df_user_rating = df_rating[df_rating['userId']==input_user_id]

        recentRatings=df_user_rating.sort_values('timestamp')['eventId'].unique()[:recent]
        df_ratings_old=df_rating.loc[df_rating['eventId'].isin(recentRatings)]

        #similar users
        df_ratings_old=df_ratings_old.loc[df_ratings_old['userId'].isin(similar_users_id)]
        df_ratings_new=df_rating.loc[df_rating['eventId'].isin(list(df_new_events['eventId']))]

        #get Recommended Events...
        df_ratings_new=df_ratings_new.loc[df_ratings_new['userId'].isin(similar_users_id)]
        df_ratings_new
        
        df_all=df_ratings_new.append(df_ratings_old,ignore_index=True)
        df_pivot=df_all.pivot(index='userId',columns='eventId',values='rating').fillna(0)
        df_pivot

        R = df_pivot.to_numpy()
        mean_ =(np.mean(R, axis = 1)).reshape(-1, 1)

        U, sigma, E = svds(R - mean_, k = 5)
        sigma = np.diag(sigma)

        dot_prod =np.dot(np.dot(U, sigma),E)
        ratings_pred = dot_prod  + mean_

        preds_matrix = pd.DataFrame(ratings_pred, columns = df_pivot.columns,index=df_pivot.index)
        # predictions = recommend_eventss(input_user_id ,preds_matrix, df_new_events, df_pivot)

        # return predictions
        preds_matrix

        # user_predictions = preds_matrix.loc[userID]
        events_pred=preds_matrix.loc[input_user_id].sort_values(ascending=False)[:10].index.to_list()

        return events_pred
    
    else:
        #Check inputs again to be sure they contain the user
        userRated=df_rating.loc[df_rating['userId']==input_user_id]
        df_old_events=df_old_events.loc[df_old_events['eventId'].isin(list(userRated['eventId']))]
        
        #new(without user rating) + old events(with user ratings)
        result=df_old_events.append(df_new_events,ignore_index=True) #get old to new.

        description = result['Description'] # Description of all
        tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 3),min_df=0, stop_words='english')
        tfidf = tf.fit_transform(description)

        cosine_similarity = linear_kernel(tfidf, tfidf)

        df_cosine_sim=pd.DataFrame(data=cosine_similarity,columns=result['eventId'],index=result['eventId'])

        # old ratings -> recent=100 by default and min rating>=4
        
        min_rating=4
        
        recent_events=df_old_events.merge(df_rating,on='eventId')
        recent_events=recent_events.loc[recent_events['rating']>=min_rating].sort_values(by='timestamp',ascending=False)['eventId'][:recent]
        
        #get df slice with predicted values
        pred=df_cosine_sim.loc[df_cosine_sim.index.isin(recent_events),list(df_new_events['eventId'])]
        
        #get top 10 predictions based on max! 0th index=>Most recommended =>9th index=>10th place on recommendation
        predictions=pred.max().sort_values(ascending=False).index[:10].to_list()
        print(predictions)
        
        return predictions
        




In [37]:
system(input_user_id,df_new_events, df_old_events,df_rating)

[232, 38, 82, 275, 320, 330, 271, 287, 306, 302]


[232, 38, 82, 275, 320, 330, 271, 287, 306, 302]